## Cleaning the data in preparation for modeling
Our goal: Figure out what makes an intersection dangerous for pedestrians and cyclists. 

<b>Approach/model 1: </b><br>
Look at the number of collisions at each intersection and quantify how much each feature of the intersection (eg, pavement, crosswalk, etc) contributes to having a high number of collisions.
Implementation: Pandas group by intersection to get number of collisions, then fit regression model on the features in the df that corresponds to characteristics of that intersection. 

Potential issues: no marker for number of people using intersection, which might be an important characteristic to note. Danger can then be defined as #collisions/#people using intersection. Some intersections with high traffic might have lower collisions rates than others.   

<b>Approach/model 2:</b><br>
Look at other features that are highly correlated with collisions. These can be independent of the intersection (weather, ligh levels, etc). While some of these cannot be improved upon by the city, others can be (eg, low light levels mitigated by street lights). There should be some grouping by intersections to see if some intersections have a disproportionate amount of collisions due to these effects. 

### Cleaning
The first part of data cleaning is to accurately measure the number of ped-car and cyclist-car collisions. Approximately 9% of the data doesn't have a ped, car, OR cyclist count, which inhibits our ability to accurately categorize accidents. 

In [127]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from pandas_profiling import ProfileReport

# Set style and settings
plt.style.use('ggplot')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 15)

In [128]:
collisions = pd.read_csv('../data/external/Collisions.csv',
                        parse_dates={'Datetime': ['INCDTTM']},
                        infer_datetime_format=True)

collisions = (
    collisions.set_index('Datetime')
    .sort_index()
    .drop(columns=['EXCEPTRSNDESC', 'EXCEPTRSNCODE', 'REPORTNO', 'STATUS'])
)

### Filling in missing data
Approximately 9% of the data is missing the persons/cars involved. We can fill this is by looking at the SDOT descriptions of the accidents

In [129]:
# How many of these involve ZERO people (ie, terrible book-keeping)
no_people = collisions.loc[(collisions['PEDCOUNT'] == 0) & 
           (collisions['PEDCYLCOUNT'] == 0) & 
           (collisions['PERSONCOUNT'] == 0) & 
            (collisions['VEHCOUNT'] == 0)]

people = collisions.loc[(collisions['PEDCOUNT'] != 0) | 
           (collisions['PEDCYLCOUNT'] != 0) | 
           (collisions['PERSONCOUNT'] != 0) | 
            (collisions['VEHCOUNT'] != 0)]

print('Fraction of data with no people involved: ', no_people.shape[0]/collisions.shape[0])

Fraction of data with no people involved:  0.08778965323268431


In [130]:
# Make dictionary with the descriptions and counts for each description
description_series = no_people['SDOT_COLDESC'].value_counts()
descriptions = list(description_series.index)
counts = list(description_series)
d = {descriptions[i]:counts[i] for i in range(len(descriptions))}
d

{'NOT ENOUGH INFORMATION / NOT APPLICABLE': 8060,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE': 4765,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END': 3963,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE SIDESWIPE': 894,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE AT ANGLE': 529,
 'MOTOR VEHICLE RAN OFF ROAD - HIT FIXED OBJECT': 306,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE SIDESWIPE': 184,
 'MOTOR VEHICLE STRUCK MOTOR VEHICLE, RIGHT SIDE AT ANGLE': 168,
 'MOTOR VEHCILE STRUCK PEDESTRIAN': 159,
 'MOTOR VEHICLE STRUCK OBJECT IN ROAD': 148,
 'MOTOR VEHICLE STRUCK PEDALCYCLIST, FRONT END AT ANGLE': 67,
 'PEDALCYCLIST STRUCK MOTOR VEHICLE FRONT END AT ANGLE': 24,
 'MOTOR VEHICLE OVERTURNED IN ROAD': 22,
 'DRIVERLESS VEHICLE RAN OFF ROAD - HIT FIXED OBJECT': 10,
 'PEDALCYCLIST STRUCK MOTOR VEHICLE LEFT SIDE SIDESWIPE': 7,
 'DRIVERLESS VEHICLE STRUCK MOTOR VEHICLE REAR END': 7,
 'PEDALCYCLIST OVERTURNED IN ROAD': 6,
 'DRIVERLESS VEHICLE STRUCK MOTOR VEHICLE FRONT EN

In [131]:
# Replace the values in no_people df and merge with original collisions set
veh_count = no_people['SDOT_COLDESC'].apply(lambda x: x.count('VEHICLE') if type(x) == str else 0)
ped_count = no_people['SDOT_COLDESC'].apply(lambda x: x.count('PEDESTRIAN') if type(x) == str else 0)
cyclist_count = no_people['SDOT_COLDESC'].apply(lambda x: x.count('PEDALCYCLIST') if type(x) == str else 0)

no_people.loc[:,'VEHCOUNT'] = veh_count
no_people.loc[:,'PEDCOUNT'] = ped_count
no_people.loc[:, 'PEDCYLCOUNT'] = cyclist_count

# Merge the people and no_people dataframes
df = pd.concat([no_people, people])

/home/maureen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### Dealing with NaNs

In [118]:
# Assigning 0/1 to binary features
df = (
    pd.get_dummies(df, columns=['SPEEDING', 'INATTENTIONIND', 'HITPARKEDCAR', 'PEDROWNOTGRNT'])
    .drop(columns=['HITPARKEDCAR_N'])
)

In [141]:
# Fixing alcohol influence
print(df['UNDERINFL'].value_counts())
df.loc[:,'UNDERINFL'] = df['UNDERINFL'].apply(lambda x: 1 if (x=='Y') or (x==1) else 0)

N    103000
0     81676
Y      5398
1      4230
Name: UNDERINFL, dtype: int64


In [ ]:
### Dealing with weather, road conditions, and light
Grouping into fewer categories

In [144]:
for col in df.columns:
    print(col, df[col].value_counts())

X -122.332653    295
-122.344896    280
-122.328079    272
-122.344997    269
-122.299160    255
              ... 
-122.379182      1
-122.361199      1
-122.288866      1
-122.331134      1
-122.282667      1
Name: X, Length: 24929, dtype: int64
Y 47.708655    295
47.717173    280
47.604161    272
47.725036    269
47.579673    255
            ... 
47.681649      1
47.539339      1
47.724955      1
47.657687      1
47.690589      1
Name: Y, Length: 24929, dtype: int64
OBJECTID 2047      1
123497    1
103015    1
105062    1
98917     1
         ..
105881    1
103832    1
126359    1
124310    1
2049      1
Name: OBJECTID, Length: 220436, dtype: int64
INCKEY 18423     1
72566     1
68468     1
78707     1
80754     1
         ..
248530    1
250577    1
205519    1
207566    1
18441     1
Name: INCKEY, Length: 220436, dtype: int64
COLDETKEY 18423     1
247852    1
233511    1
235558    1
229413    1
         ..
59575     1
57526     1
63669     1
61620     1
18441     1
Name: COLDETKEY,

In [143]:
# Pickle the dataframe AND save as csv
df.to_pickle('../data/processed/fixed_empty_ped_df.pkl')
df.to_csv('../data/processed/fixed_empty_ped.csv')